In [2]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from torchvision.transforms import transforms
import matplotlib as plt
print(torch.__version__)
print(plt.__version__)

2.9.1+cu128
3.10.8


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
# Hyperparams
BATCH_SIZE = 128  # according to the paper

In [5]:
# CIFAR10
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
    transforms.RandomHorizontalFlip()  # p=0.5 by default
])

train = datasets.CIFAR10("../data", train=True, download=True, transform=transform)
test = datasets.CIFAR10("../data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

100.0%


In [ ]:
# MNIST
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train = datasets.MNIST("../data", train=True, download=True, transform=transform) # 28x28 resolution
test = datasets.MNIST("../data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# architecture
class ResNetBlock(nn.Module):
  def __init__(self):
    self.conv1 = nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=0)
